In [1]:
# Import the libraries
import finpie as fp
import pandas as pd
import yfinance as yf
import pandas as pd

### Helper Functions

In [34]:
def get_date(ticker: str):

    # get the income statement
    inc_st = fp.Fundamentals(ticker, source = 'yahoo')
    df_inc_st = inc_st.income_statement()

    # Change the values to original
    # --------------------------------------------------------------------------
    exclude_column = ['basic_eps', 'diluted_eps']

    # make the data-frame
    df_inc_st_org = df_inc_st.loc[:, ~df_inc_st.columns.isin(exclude_column)]

    # change to the same name for easyness
    df_inc_st = df_inc_st_org.copy()

    # Multiply the df with 1000 to make original
    df_inc_st = df_inc_st.mul(1000)

    # --------------------------------------------------------------------------
    """
    We will use 'yfinance' package to get the balance sheet data as the 
    previous libaray 'finpie' doesn't gave the broken down results.
    """

    # Balance Statement
    tgt = yf.Ticker(ticker)
    df_bal_st = tgt.get_balance_sheet()

    # Combine the Balance Sheet and Income Statement
    # --------------------------------------------------------------------------
    
    # Pivot the 'df_inc_st'
    df_inc_st_t = df_inc_st.transpose()

    # ----------------------------------
    # Change the column name (inc st)
    new_col = df_inc_st_t.columns.year

    # Assign the new column (inc st)
    df_inc_st_t.columns = new_col


    # Change the column name (bal st)
    new_col = df_bal_st.columns.year

    # Assign the new column (bal st)
    df_bal_st.columns = new_col


    # ----------------------------------

    # # Change the column name
    # df_inc_st_t.columns = ['2022', '2021', '2020', '2019']

    # # Change the column name for 'df_bal_st'
    # df_bal_st.columns = ['2022', '2021', '2020', '2019']

    # Combine the 'df_inc_st_t' and 'df_bal_st'
    df_comb = pd.concat([df_inc_st_t, df_bal_st], axis = 0)

    # Transposing the df to get correct format
    df = df_comb.transpose()

    # --------------------------------------------------------------------------
    # Financial Ratios Calculations

    # Gross Profit Margin
    df['gross_profit_margin'] = (df['gross_profit']/df['total_revenue'])*100

    # Operating Margin
    df['operating_margin'] = (df['operating_income']/df['total_revenue'])*100

    # Net Profit Margin
    df['net_profit_margin'] = (df['normalized_income']/df['total_revenue'])*100

    # Debt-to-equity Ratio
    df['debt_equity_ratio'] = df['Total Liab']/df['Total Stockholder Equity']

    # Current Ratio
    df['current_ratio'] = df['Total Current Assets']/df['Total Current Liabilities']

    # Fixed Asset Turnover
    df['fixed_asset_turn'] = df['total_revenue']/df['Property Plant Equipment']

    # Total Asset Turnover
    df['total_asset_turn'] = df['total_revenue']/df['Total Assets']

    # Return on Asset
    df['ROA'] = df['net_profit_margin']*df['total_asset_turn']

    # Return on Equity
    df['ROE'] = df['normalized_income'] / df['Total Stockholder Equity']

    # --------------------------------------------------------------------------

    return df


In [61]:
# Combine multiple ticker

def combine_ticker(ticker_list: list):

    df = pd.DataFrame()

    for items in ticker_list:

        df1 = get_date(items)
        df1.sort_index(ascending=True, inplace=True)
        df1.reset_index(inplace=True)
        df1['company'] = items
        df1 = pd.melt(df1, id_vars=['index', 'company'])

        df = pd.concat([df, df1], axis=0, ignore_index=True)
    
    df['index'] = pd.to_datetime(df['index'], format = '%Y')
    
    return df

In [74]:
ticker_list = ['GOOG', 'AMZN']

df_comb = combine_ticker(ticker_list = ticker_list)

In [79]:
# Plot ROA
from tkinter.tix import InputOnly


ratio_type = 'gross_profit_margin'
ratio_title = 'Return on Asset (ROA)'


df_roa = df_comb[df_comb['variable'] == ratio_type]
df_roa.dropna(inplace=True)
df_roa['value'] = pd.to_numeric(df_roa['value'])
df_roa = df_roa.round(2)

import plotly.express as px
fig = px.line(df_roa, x= 'index', y = 'value', color='company', markers=True,
                labels = {"value": ratio_type, "index": 'Year'},
                title= ratio_title,
                template='plotly_white'
                )
# fig.update_layout(title_text = 'This is subtitle')
fig.show()

C:\Users\Sande\AppData\Local\Temp\ipykernel_12124\2313752880.py:2: DeprecationWarning:

The Tix Tk extension is unmaintained, and the tkinter.tix wrapper module is deprecated in favor of tkinter.ttk

C:\Users\Sande\AppData\Local\Temp\ipykernel_12124\2313752880.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Sande\AppData\Local\Temp\ipykernel_12124\2313752880.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [71]:
df_roa

,index,company,variable,value
256,2018-01-01,GOOG,ROA,12.636551
257,2019-01-01,GOOG,ROA,12.068882
258,2020-01-01,GOOG,ROA,11.033123
259,2021-01-01,GOOG,ROA,18.357288
500,2018-01-01,AAPL,ROA,NaN
501,2019-01-01,AAPL,ROA,16.32301
502,2020-01-01,AAPL,ROA,17.725572
503,2021-01-01,AAPL,ROA,26.974205
768,2018-01-01,AMZN,ROA,6.193129
769,2019-01-01,AMZN,ROA,5.144552


In [88]:
import plotly.express as px
def make_plot(ratio_name, df, ratio_full_name):
    """
    ratio_name: Name of the ratio plot as string. This has to be same as the names
                in the dataframe or imported data.
    df: the combined data-frame that has main data
    ratio_full_name: full name of the ratio. This will be the title of plot

    """
    # Subsetting the data-frame
    df_sub = df[df['variable'] == ratio_name]

    # Dropping the NAs
    df_sub.dropna(inplace=True)

    # Changing the values to integer from string
    df_sub['value'] = pd.to_numeric(df_sub['value'])

    # Rounding the values
    df_sub = df_sub.round(2)

    # Plot

    fig = px.line(df_sub, x= 'index', y = 'value', color='company', markers=True,
                labels = {"value": ratio_name, "index": 'Year'},
                title= ratio_full_name,
                template='plotly_white'
                )
    fig.show()

    

In [89]:
make_plot('gross_profit_margin', df_comb, 'Return on Asset')

C:\Users\Sande\AppData\Local\Temp\ipykernel_12124\304986341.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Sande\AppData\Local\Temp\ipykernel_12124\304986341.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

